In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import pylab as pl
import PIL
from PIL import Image
import os
import base64
from StringIO import StringIO

from sklearn.decomposition import RandomizedPCA
from sklearn.neighbors import KNeighborsClassifier

import os
print os.listdir("./FINAL_PICTURES/")[:5] # just print first 5 images to see filename structure
print len(os.listdir("./FINAL_PICTURES/")) 

['000001_1.jpg', '000002_3.jpg', '000003_2.jpg', '000004_4.jpg', '000005_2.jpg']
128


In [2]:
#setup a standard image size; this will distort some images but will get everything into the same shape
STANDARD_SIZE = (100, 100)
def img_to_matrix(filename, verbose=False):
    """
    takes a filename and turns it into a numpy array of RGB pixels
    """
    img = PIL.Image.open(filename)
    if verbose==True:
        print "changing size from %s to %s" % (str(img.size), str(STANDARD_SIZE))
    img = img.resize(STANDARD_SIZE)
    img = list(img.getdata())
    img = map(list, img)
    img = np.array(img)
    return img

def flatten_image(img):
    """
    takes in an (m, n) numpy array and flattens it 
    into an array of shape (1, m * n)
    """
    s = img.shape[0] * img.shape[1]
    img_wide = img.reshape(1, s)
    return img_wide[0]

In [3]:
# TODO PATH TO YOUR DATA
img_dir = "./FINAL_PICTURES/"
images = [img_dir+ f for f in os.listdir(img_dir)]
labels = [l[-5:-4] for l in images]

data = []
for image in images:
    img1 = img_to_matrix(image)
    img = flatten_image(img1)
    data.append(img.tolist())
data = np.asarray(data)

In [4]:
print labels
print data[:5]

['1', '3', '2', '4', '2', '4', '4', '1', '2', '2', '3', '3', '1', '2', '4', '1', '1', '4', '3', '2', '2', '3', '1', '3', '3', '2', '3', '1', '4', '4', '2', '1', '4', '2', '4', '4', '3', '4', '3', '2', '4', '4', '1', '1', '3', '2', '4', '4', '2', '1', '4', '2', '1', '4', '3', '2', '3', '4', '4', '2', '4', '3', '4', '4', '1', '1', '3', '4', '1', '2', '4', '1', '4', '3', '3', '2', '3', '2', '3', '3', '1', '2', '3', '4', '3', '1', '2', '3', '2', '3', '1', '3', '2', '4', '3', '2', '4', '3', '2', '4', '1', '3', '4', '1', '4', '4', '1', '3', '3', '2', '2', '1', '3', '3', '2', '1', '3', '3', '2', '3', '4', '4', '4', '2', '3', '2', '2', '4']
[ [33, 36, 41, 35, 38, 43, 39, 43, 46, 40, 44, 47, 39, 42, 47, 38, 41, 46, 35, 38, 43, 37, 40, 45, 38, 41, 46, 38, 41, 46, 41, 44, 49, 45, 48, 53, 55, 58, 63, 44, 49, 53, 52, 57, 63, 47, 54, 60, 48, 55, 61, 57, 62, 68, 55, 62, 70, 61, 68, 74, 83, 97, 100, 64, 69, 75, 66, 73, 81, 77, 86, 95, 76, 85, 94, 76, 86, 95, 75, 86, 92, 74, 85, 91, 82, 95, 101, 87, 10

In [5]:
# split the images into training and testing data
# randomly select approximately 70% of data for training
trainFeatures = []
trainLabels = []
testFeatures = []
testLabels = []
trainFeatures1 = []
trainLabels1 = []
testFeatures1 = []
testLabels1 = []

is_train = np.random.uniform(0, 1, len(data)) <= 0.7 # this creates an array of booleans
print len(is_train)
print len([i for i in is_train if i==True]) # checking the number of training samples
print len([i for i in is_train if i==False]) # checking the number of test samples

for i in range(len(is_train)):
    if is_train[i]==True:
        trainFeatures1.append(data[i])
        trainLabels1.append(labels[i])
    else:
        testFeatures1.append(data[i])
        testLabels1.append(labels[i])
trainFeatures = np.asarray(trainFeatures1)
trainLabels = np.asarray(trainLabels1)
testFeatures = np.asarray(testFeatures1)
testLabels = np.asarray(testLabels1)

128
95
33


In [6]:
print len(trainFeatures),len(trainLabels) # checking the number of training samples
print len(testFeatures),len(testLabels) # checking the number of test samples
print trainFeatures[:2]
print testFeatures[:2]

95 95
33 33
[ [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 254, 255, 255, 255, 254, 255, 255, 255, 253, 255, 254, 255, 255, 255, 253, 255, 255, 253, 255, 255, 253, 254, 255, 253, 255, 255, 253, 255, 255, 253, 255, 255, 255, 255, 254, 255, 255, 254, 255, 255, 254, 255, 255, 255, 255, 255, 254, 255, 255, 255, 255, 255, 255, 253, 255, 254, 255, 255, 255, 253, 255, 254, 255, 255, 255, 253, 255, 255, 255, 254, 255, 253, 255, 255, 251, 255, 254, 253, 255, 255, 253, 255, 255, 253, 255, 255, 253, 255, 255, 253, 255, 254, 

In [7]:
pca = RandomizedPCA(n_components=5)
train_x = pca.fit_transform(trainFeatures)
test_x = pca.transform(testFeatures)

ValueError: setting an array element with a sequence.